In [182]:
import pandas as pd
import sqlite3

con = sqlite3.connect("database.sqlite")
#d_Country = pd.read_sql("SELECT * FROM Country", con)
#d_League = pd.read_sql("SELECT * FROM  League", con)
d_Match = pd.read_sql("SELECT * FROM Match", con)
d_Player = pd.read_sql("SELECT * FROM Player", con)
d_Player_Stats = pd.read_sql("SELECT * FROM Player_Stats", con)
d_Team = pd.read_sql("SELECT * FROM Team", con)
#d_sqlite_sequence = pd.read_sql("SELECT * FROM sqlite_sequence", con)

con.close()

In [183]:
X = []
#пихаем ставки и ids команд
X = pd.concat(
   [
    d_Match['away_team_api_id'], d_Match['home_team_api_id'],
    d_Match['B365H'],d_Match['B365D'],d_Match['B365A'],d_Match['BWH'],d_Match['BWD'],d_Match['BWA'],d_Match['IWH'],d_Match['IWD'],d_Match['IWA'],d_Match['LBH'],d_Match['LBD'],d_Match['LBA'],d_Match['WHH'],d_Match['WHD'],d_Match['WHA'],d_Match['SJH'],d_Match['SJD'],d_Match['SJA'],d_Match['VCH'],d_Match['VCD'],d_Match['VCA'],d_Match['GBH'],d_Match['GBD'],d_Match['GBA'],d_Match['BSH'],d_Match['BSD'],d_Match['BSA'],
    ],
axis = 1)
print len(X)
X = X[:23000]
X = X.fillna(1)

#X = X.dropna(axis=0, how='all',subset =[i for i in X.columns[-30:]])


import numpy as np
d_Results = d_Match[["home_team_goal","away_team_goal"]]
def winner(a, b):
    if a > b:
        return 1
    elif a < b:
        return -1
    else:
        return 0
y = []
for i in range(len(X)):
    win = winner(d_Results.iloc[i][0], d_Results.iloc[i][1])
    y.append(win)
#for i in range(len(X)):
#    if y[i] == 0:
#        del y[i]
import numpy as np
y = np.array(y)



25945


In [184]:
#усредняем ставки
def isNaN(num):
    return num != num
X3 = []
for i in range(len(X)):
    tmp = d_Match.iloc[i]
    draw_sum = home_sum = away_sum = 0
    if not isNaN(tmp['B365H']) and not isNaN(tmp['B365A']) and not isNaN(tmp['B365D']):
        home_sum += 0 if isNaN(tmp['B365H']) else tmp['B365H']
        away_sum += 0 if isNaN(tmp['B365A']) else tmp['B365A']
        draw_sum += 0 if isNaN(tmp['B365D']) else tmp['B365D']
    if not isNaN(tmp['BWH']) and not isNaN(tmp['BWA']) and not isNaN(tmp['BWD']):
        home_sum += 0 if isNaN(tmp['BWH']) else tmp['BWH']
        away_sum += 0 if isNaN(tmp['BWA']) else tmp['BWA']
        draw_sum += 0 if isNaN(tmp['BWD']) else tmp['BWD']
    if not isNaN(tmp['IWH']) and not isNaN(tmp['IWA']) and not isNaN(tmp['IWD']):
        home_sum += 0 if isNaN(tmp['IWH']) else tmp['IWH']
        away_sum += 0 if isNaN(tmp['IWA']) else tmp['IWA']
        draw_sum += 0 if isNaN(tmp['IWD']) else tmp['IWD']
    if not isNaN(tmp['LBH']) and not isNaN(tmp['LBA']) and not isNaN(tmp['LBD']):
        home_sum += 0 if isNaN(tmp['LBH']) else tmp['LBH']
        away_sum += 0 if isNaN(tmp['LBA']) else tmp['LBA']
        draw_sum += 0 if isNaN(tmp['LBD']) else tmp['LBD']
    if not isNaN(tmp['WHH']) and not isNaN(tmp['WHA']) and not isNaN(tmp['WHD']):
        home_sum += 0 if isNaN(tmp['WHH']) else tmp['WHH']
        away_sum += 0 if isNaN(tmp['WHA']) else tmp['WHA']
        draw_sum += 0 if isNaN(tmp['WHD']) else tmp['WHD']
    if not isNaN(tmp['SJH']) and not isNaN(tmp['SJA']) and not isNaN(tmp['SJD']):
        home_sum += 0 if isNaN(tmp['SJH']) else tmp['SJH']
        away_sum += 0 if isNaN(tmp['SJA']) else tmp['SJA']
        draw_sum += 0 if isNaN(tmp['SJD']) else tmp['SJD']
    if not isNaN(tmp['VCH']) and not isNaN(tmp['VCA']) and not isNaN(tmp['VCD']):
        home_sum += 0 if isNaN(tmp['VCH']) else tmp['VCH']
        away_sum += 0 if isNaN(tmp['VCA']) else tmp['VCA']
        draw_sum += 0 if isNaN(tmp['VCD']) else tmp['VCD']
    if not isNaN(tmp['GBH']) and not isNaN(tmp['GBA']) and not isNaN(tmp['GBD']):
        home_sum += 0 if isNaN(tmp['GBH']) else tmp['GBH']
        away_sum += 0 if isNaN(tmp['GBA']) else tmp['GBA']
        draw_sum += 0 if isNaN(tmp['GBD']) else tmp['GBD']
    if not isNaN(tmp['BSH']) and not isNaN(tmp['BSA']) and not isNaN(tmp['BSD']):
        home_sum += 0 if isNaN(tmp['BSH']) else tmp['BSH']
        away_sum += 0 if isNaN(tmp['BSA']) else tmp['BSA']
        draw_sum += 0 if isNaN(tmp['BSD']) else tmp['BSD']
    X3.append([home_sum / 9.0, draw_sum / 9.0, away_sum / 9.0, tmp['home_team_api_id'], tmp['away_team_api_id']])

In [185]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score


X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3)
max_acc_t = 0
max_acc_k = 0
max_acc_l = 0
max_acc_f = 0
for depth in range(7, 16):
    tree = DecisionTreeClassifier(max_depth=depth)
    tree.fit(X_train,Y_train)
    y_pred = tree.predict(X_test)
    acc = accuracy_score(Y_test,y_pred)
    max_acc_t = max(acc, max_acc_t)
for n in range(1, 7):
    tree = KNeighborsClassifier(n_neighbors=n)
    tree.fit(X_train,Y_train)
    y_pred = tree.predict(X_test)
    acc = accuracy_score(Y_test,y_pred)
    max_acc_k = max(acc, max_acc_k)
tree = LogisticRegression()
tree.fit(X_train,Y_train)
y_pred = tree.predict(X_test)
acc = accuracy_score(Y_test,y_pred)
max_acc_l = max(acc, max_acc_l)

for depth in range(7, 16):
    tree = RandomForestClassifier(n_estimators=50, max_depth=depth)
    tree.fit(X_train,Y_train)
    y_pred = tree.predict(X_test)
    acc = accuracy_score(Y_test,y_pred)
    max_acc_f = max(acc, max_acc_f)


print max_acc_t
print max_acc_k
print max_acc_l
print max_acc_f

0.510579710145
0.427536231884
0.519565217391
0.519275362319
